In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

In [2]:
from IPython.display import Image

In [3]:
r = requests.get('https://www.eluniversal.com/')

In [4]:
r.encoding='utf-8'

In [5]:
s = BeautifulSoup(r.text,'html.parser')

In [6]:
print(s.prettify())

<!DOCTYPE html>
<html class="no-js" lang="es">
 <head>
  <title>
   EL UNIVERSAL
  </title>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
  <!-- <script src="https://ajax.googleapis.com/ajax/libs/jquery/2.1.4/jquery.min.js"></script> -->
  <!-- INICIO MODIFICACIONES TEMPORALES -->
  <form action="buscador" enctype="application/x-www-form-urlencoded" id="busq-index" method="post">
   <input id="index-query" name="query" style="display: none" type="text"/>
   <input id="index-pagina" name="pagina" style="display: none" type="text" value="1"/>
   <input id="index-seccioncod" name="seccioncod" style="display: none" type="text"/>
  </form>
  <script>
   function buscarIndex(seccioncod){
            $('#index-seccioncod').val(seccioncod);
            $('#busq-index').submit();
        }
  </script>
  <!-- FIN MODIFICACIONES TEMPORALES -->
  <script>
   console.log('--> srvapache002.eluniversal.com/01-HOME/V01-D13B15v05/943

In [7]:
link = s.find_all('a',attrs={'class':'title'})
links = [i.get('href') for i in link]            #Recojo los links de las noticias

In [9]:
len(links)

72

In [8]:
print(links[69])

http://www.eluniversal.com/economia/90321/un-concurso-para-celebrar-el-mes-mas-romantico-del-ano


In [9]:
def news(links):
    
    try:
        all_data = []
        
        
        for idx , link in enumerate(links):
            print('Scraping {} de {}'.format(idx,len(links)-1))
            r = requests.get(link)
            
            if r.status_code == 200:
                s = BeautifulSoup(r.text,'lxml')

                titular = s.find('h2',attrs={'class':'title'})
                titular = titular.a.get_text().strip()
                
                body = s.find('div',attrs={'class':"note-text"})
                body = body.get_text()
                
                try:
                    img = s.find('figure',attrs={'class':"thumb img-bg"}).find_all('img')
                    img = img[-1].get('src')
                    
                    
                    r_img = requests.get(img)

                    if r_img.status_code == 200:
                        img = r_img.content
                
                except Exception as e:
                    print(e)
                
                data = {
                    'titular':titular,
                    'body':body,
                    'image':img
                }
                
                all_data.append(data)
                
        
            
            else:
                return None 
        return all_data
    
    except Exception as e:
        print('-' * 5, 'E R R O R --- P U T I T O')
        print(e)
        

In [10]:
data_test = news(links)

Scraping 0 de 71
Scraping 1 de 71
Scraping 2 de 71
Scraping 3 de 71
Scraping 4 de 71
Scraping 5 de 71
Scraping 6 de 71
Scraping 7 de 71
Scraping 8 de 71
Scraping 9 de 71
Scraping 10 de 71
Scraping 11 de 71
Scraping 12 de 71
Scraping 13 de 71
Scraping 14 de 71
Scraping 15 de 71
Scraping 16 de 71
Scraping 17 de 71
Scraping 18 de 71
Scraping 19 de 71
Scraping 20 de 71
Scraping 21 de 71
Scraping 22 de 71
Scraping 23 de 71
Scraping 24 de 71
Scraping 25 de 71
Scraping 26 de 71
Scraping 27 de 71
Scraping 28 de 71
Scraping 29 de 71
Scraping 30 de 71
Scraping 31 de 71
Scraping 32 de 71
Scraping 33 de 71
Scraping 34 de 71
Scraping 35 de 71
Scraping 36 de 71
Scraping 37 de 71
Scraping 38 de 71
Scraping 39 de 71
Scraping 40 de 71
Scraping 41 de 71
Scraping 42 de 71
Scraping 43 de 71
Scraping 44 de 71
Scraping 45 de 71
Scraping 46 de 71
Scraping 47 de 71
Scraping 48 de 71
Scraping 49 de 71
Scraping 50 de 71
Scraping 51 de 71
Scraping 52 de 71
Scraping 53 de 71
Scraping 54 de 71
Scraping 55 de 71
Sc